In [ ]:
import os
import pandas as pd
import geopandas as gpd
from tqdm import tqdm

In [ ]:
geo_folder = r'C:\Users\CrickB\OneDrive - Version 1\Documents\Repositories\data\UK_geography'

LPA_geojson = os.path.join(geo_folder, r'Local_Planning_Authorities_May_2021_UK_BFE_2022_8497080629868369416.geojson')

census_folder = r'C:\Users\CrickB\OneDrive - Version 1\Documents\Repositories\data\census_2021\census2021-ts008'

area_type = ''
# ctry, lsoa, ltla, msoa, oa, rgn, ulta

census_geojson = ''

In [ ]:
census_data_csv = os.path.join(census_folder, 'census2021-ts008-msoa.csv')

census_df = pd.read_csv(census_data_csv)

census_df = census_df.drop(columns=[
    'Sex: Female; measures: Value',
    'Sex: Male; measures: Value'
]).rename(columns={
    'Sex: All persons; measures: Value': 'population'
})

census_df.head()

In [ ]:
oa_geojson = r"C:\Users\CrickB\OneDrive - Version 1\Documents\Repositories\data\UK_geography\MSOA_2021_EW_BFE_V7.geojson"

oa_gdf = gpd.read_file(oa_geojson)

oa_gdf.head()

In [ ]:
oa_pop_gdf = gpd.GeoDataFrame(
    data=pd.merge(
        left=oa_gdf,
        right=census_df,
        left_on='MSOA21CD',
        right_on='geography code',
        how='inner'
    ),
    geometry='geometry',
    crs=oa_gdf.crs
).drop(columns=['geography', 'geography code'])

oa_pop_gdf.head()

In [ ]:
oa_pop_geojson = r"C:\Users\CrickB\OneDrive - Version 1\Documents\Repositories\data\census_2021_geography\MSOA_2021_EW_BFE_V7_population.geojson"

oa_pop_gdf.to_file(oa_pop_geojson)

In [ ]:
LPA_gdf = gpd.read_file(LPA_geojson)

LPA_gdf.head()

In [ ]:
LPA_gdf.to_crs('EPSG:4326', inplace=True)
oa_pop_gdf.to_crs('EPSG:4326', inplace=True)

In [ ]:
LPA_gdf['LPA_area'] = LPA_gdf.geometry.area

In [ ]:
oa_pop_gdf['MSOA_area'] = oa_pop_gdf.geometry.area

In [ ]:
intersection_gdf = gpd.overlay(
    df1=LPA_gdf,
    df2=oa_pop_gdf,
    how='intersection'
)

In [ ]:
intersection_gdf['intersection_area'] = intersection_gdf.geometry.area

In [ ]:
intersection_gdf.head()

In [ ]:
LPA_gdf['LPA_population'] = 0

In [ ]:
for LPA_row in tqdm(LPA_gdf.itertuples(), total=len(LPA_gdf)):

    LPA_intersection_gdf = intersection_gdf.loc[intersection_gdf['LPA21NM']==LPA_row.LPA21NM]

    LPA_intersection_gdf['LPA_population'] = LPA_intersection_gdf['intersection_area'] / LPA_intersection_gdf['MSOA_area'] * LPA_intersection_gdf['population']

    LPA_population = LPA_intersection_gdf['LPA_population'].sum()

    LPA_gdf.at[LPA_row.Index, 'LPA_population'] = LPA_population

In [ ]:
LPA_gdf.head()

In [ ]:
print(f'LPA total population: {LPA_gdf["LPA_population"].sum()}')
print(f'MSOA total population: {oa_pop_gdf["population"].sum()}')

In [ ]:
LPA_population_geojson = r'C:\Users\CrickB\OneDrive - Version 1\Documents\Repositories\data\census_2021_geography\LPA_census_2021_population.geojson'

LPA_gdf.to_file(LPA_population_geojson)

In [ ]:
LPA_gdf_27700 = LPA_gdf.to_crs('EPSG:27700')

LPA_gdf_27700.crs

In [ ]:
LPA_gdf_27700['LPA_area'] = LPA_gdf_27700.geometry.area / 1000000

LPA_gdf_27700.rename(columns={'LPA_area': 'LPA_area_km2'}, inplace=True)

LPA_gdf_27700.head()

In [ ]:
LPA_gdf = LPA_gdf_27700.to_crs('EPSG:4326')

LPA_gdf.head()

In [ ]:
LPA_gdf.to_file(LPA_population_geojson)